# SCALES low-res SED example

This notebook runs through all the steps for generating a raw SCALES image using either:

(1) a target spectrum in erg/s/cm^2/um + a PSF
OR
(2) a model datacube with fluxes in erg/s/cm^2/um

You don't need to change any of the configuration stuff at the beginning if you don't want to. There are comments that describe some parameters that you might be interested in adjusting.

Right now we just have the 2-5 um mode enabled. In the future you'll be able to switch to other low-res modes. To do that all you have to do is change arg_spaxel['min_wavelength'] and arg_spaxel['max_wavelength'] at the bottom of the "Configure SCALES Mode" section.

This example just uses PHOENIX models to set up simple stellar targets as "Target" objects. It then uses PHOENIX models to make an example model datacube that you can feed to the simulator. You can replace that with whatever scene you like, as long as the dimensions match the number of spaxels in each dimension (108 by 108). 

In the end it does a simple aperture extraction to generate a datacube. 

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits
import scipy.ndimage
from astropy import units as u

from scalessim.DFT import *
from scalessim.binning import *
from scalessim.phasescreen import *
from scalessim.pupil import *
from scalessim.io import *
from scalessim.focal_plane import *
from scalessim.targs import *
from scalessim.base import *
from scalessim.helpers import *

## Configure your H2RG

Load in h2rg configuration file 

In [2]:
import configparser
from detector import nghxrg as ng

config = configparser.ConfigParser()
config.read('h2rg.ini')

arg_detector = {}
arg_detector.update(read_ini(config['Defined']))

ng_h2rg = ng.HXRGNoise(verbose=False,
                        wind_mode='WINDOW',
                        naxis1=1024, naxis2=1024,
                        pca0_file='./detector/lmircam_pca0.fits')


/Users/stephsallum/Dropbox/scalessim/detector/nghxrg.py:345: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if mode is 'pink':


Define some reasonable noise parameters

In [3]:
rd_noise=arg_detector['readout_noise_cds']*1.   # White read noise per integration
pedestal=arg_detector['pedestal']*1.   # DC pedestal drift rms
c_pink=arg_detector['c_pink']*1.     # Correlated pink noise
u_pink=arg_detector['u_pink']*1.     # Uncorrelated pink noise
acn=arg_detector['acn']*1.        # Correlated ACN
pca0_amp=arg_detector['pca0_amp']*1.   # Amplitude of PCA zero "picture frame" noise

#arg_detector 

## Configure SCALES mode

Read in SCALES config file for H2RG

In [4]:
config.read('scales_h2rg.ini')
arg_spaxel = {}
arg_spaxel.update(read_ini(config['Defined']))
arg_spaxel.update(read_ini(config['Derived']))
arg_spaxel.update(read_ini(config['User']))

#arg_spaxel

Adjust minimum and maximum wavelength if we want to

In [5]:
arg_spaxel['min_wavelength'] = 2.0 #minimum wavelength in microns
arg_spaxel['max_wavelength'] = 5.2 #maximum wavelength in microns

## Set up lenslet traces

In [6]:
lens = Lenslet(arg_spaxel)
lens.get_shifts()
lens.make_trace(phys=False)

## Read in Keck pupil

In [8]:
keck = fits.getdata('keckpupil.fits')
keck = scipy.ndimage.zoom(keck, 10, order=0)
pupil = Pupil()
pupil.pupil = keck

## Filtered phase screen for toy AO correction

Can set screen1 to zeros to leave this out

In [9]:
screen = PhaseScreen(pupil=pupil, L0=10)
screen.sr_factor = screen.set_strehl(.8, 500e-9)
screen1 = screen.get_res()
#screen1 = np.zeros(screen1.shape)

## Use pupil and residual phase to generate cube of Keck PSFs

In [10]:
nspax = arg_spaxel['no_spaxel']
nlam = lens.trace.shape[0]

keck_cube = np.zeros((nlam, nspax, nspax))
mag1 = magn(lens.lmin)
mag2 = magn(lens.lmax)

psf_file = 'keck_psf_cube_'+str(lens.lmin)+'_'+str(lens.lmax)+'_n'+str(nlam)+'.fits'
if os.path.isfile(psf_file)==False:
    for i, mag in enumerate(np.linspace(mag1, mag2, nlam)):
        W = DFT_matrix(1600, mag)
        A = pupil.pupil * np.exp(1.j * screen1)
        out = abs(W.dot(A).dot(W))**2
        out /= np.sum(out, axis=(0,1))
        keck_cube[i] = frebin(out, (108, 108))
    fits.writeto(psf_file,np.array(keck_cube),clobber=True)
else: keck_cube = pyfits.getdata(psf_file)

## Define some observational parameters

-Reads in sky transmission and emission models provided by Gemini for MK site: https://www.gemini.edu/observing/telescopes-and-sites/sites#IRSky

-Reads in atmospheric dispersion data based on relative humidity, temperature, pressure, and airmass

-Instrument transmission & emission includes SCALES + AO emissivity

-Default QE is 70%

-For now set up perfect filter with tophat from lmin to lmax

-Define single frame integration time in seconds

In [11]:
vapor = 1 #PWV in mm
airmass = 1 
skybg = SkyBG(vapor,airmass)
skytrans = SkyTrans(vapor,airmass)
atmodisp = AtmoDispersion(90,20,600)
inst_emissivities = [.08]*3 + [.01]*8
inst_temps = [273*u.K]*11
inst = InstTransEm(inst_emissivities, inst_temps)
qe = QE()
filt = Filter(lens.lmin,lens.lmax)
dit = 1 ###integration time in seconds

In [ ]:
####organize all these and pass to focal_plane 
args_new = {'Lenslet_object':lens,
            'SkyBG':skybg,
            'SkyTrans':skytrans,
            'InstTransEm':inst,
            'Filter':filt,
            'QE':qe}
fp = FocalPlane(args_new)

## Setting up a target for scalessim

In [ ]:
T_s = 10000 #stellar temperature in K
logg = 4.5 #log surface gravity [cgs]
zz = 0.0 #metallicity 
rstar = 2.0 #stellar radius in solar radii
dstar = 100.0 #distance in pc

wav,I_lam = phoenix_star(T_s,logg,zz,rstar,dstar)

####set this up as a Target
targ = Target(wav,I_lam)

## Generate raw SCALES frame using target + PSF

In [ ]:
raw, bg_star_truth = fp.get_fp(dit*u.s,targ,return_full=True,PSF=keck_cube)
rawfile = 'raw_frame_targ.fits'
pyfits.writeto(rawfile,np.array(raw),clobber=True)

f = plt.figure(figsize=(5,5))
plt.imshow(raw**0.25)
plt.show()

## Feeding scalessim a datacube instead

Start with PSF and stellar spectrum

In [ ]:
scene_cube = keck_cube.copy()

T_s = 10000 #stellar temperature in K
logg = 4.5 #log surface gravity [cgs]
zz = 0.0 #metallicity 
rstar = 2.0 #stellar radius in solar radii
dstar = 100.0 #distance in pc

wav,I_lam = phoenix_star(T_s,logg,zz,rstar,dstar)
I_lam2 = spectres(fp.lam.value,wav,I_lam)

for x in range(len(I_lam2)): scene_cube[x]*=I_lam2[x]

Shifted PSF for companion - fix this to actually re-generate the PSF later

In [ ]:
keck_cube_2 = shift(keck_cube,(0,20,20))

scene_cube_2 = keck_cube_2.copy()
T_s = 10000 #stellar temperature in K
logg = 4.5 #log surface gravity [cgs]
zz = 0.0 #metallicity 
rstar = 2.0 #stellar radius in solar radii
dstar = 100.0 #distance in pc

wav,I_lam = phoenix_star(T_s,logg,zz,rstar,dstar)
I_lam2 = spectres(fp.lam.value,wav,I_lam)

for x in range(len(I_lam2)): scene_cube_2[x]*=I_lam2[x]

In [ ]:
scene = scene_cube + scene_cube_2
plt.imshow(scene[0])
plt.show()

## Generate a raw SCALES frame using input cube

In [ ]:
scene = scene*u.erg / u.cm**2 / u.micron / u.s
raw_c, bg_star_truth = fp.get_fp(dit*u.s,cube=scene,return_full=True)
rawfile = 'raw_frame_cube.fits'
pyfits.writeto(rawfile,np.array(raw_c),clobber=True)

f = plt.figure(figsize=(5,5))
plt.imshow(raw_c**0.2)
plt.show()

## Cube up raw frame using simple aperture extraction

In [ ]:
from scipy import sparse

nxlam = int(arg_spaxel['spectra_length'])
xlams = np.linspace(lens.lmin,lens.lmax,nxlam)
rectmat = sparse.load_npz('2.0_5.2_rectmat.npz')

raw_f = raw_c.flatten()
cube = (rectmat*raw_f).reshape(nxlam,nspax,nspax)

f = plt.figure(figsize=(10,3))
f.add_subplot(131)
plt.imshow(cube[0])
plt.title(str(np.round(xlams[0],1))+' um')
f.add_subplot(132)
plt.imshow(cube[22])
plt.title(str(np.round(xlams[22],1))+' um')
f.add_subplot(133)
plt.imshow(cube[50])
plt.title(str(np.round(xlams[50],1))+' um')
plt.show()